# Displaying webcam video in IPython notebook at (relatively) high framerate

When working on my project I realized, I can use OpenCV in Python to grap image from webcam as Numpy array, modify it and then display it using OpenCV's **cv2.imshow()**. OpenCV will create a window and push my frame there. However, this will not work in a IPython notebook. I found few solutions to implement the same functionality, but they all were slow (about 250 ms per frame).

Here, I combine and modify these two examples to get achieve about 5 times higher framerate:
1. __[Showing webcame image using OpenCV and matplotlib](https://gist.github.com/tibaes/35b9dbd7cbf81a98955067aa318290e7#file-video)__. Important moment here is that previous frame is cleared from screen using **IPython.display.clear_output()**
2. __[Minimal code for rendering a numpy array as an image in a Jupyter notebook in memory](https://gist.github.com/kylemcdonald/2f1b9a255993bf9b2629)__. It uses PIL to convert NumPy array to .PNG format in order to display it with **IPython.display.display()**

Both are relatively slow. The slowest step in the first one is the __[matplotlib.pyplot.imshow()](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.imshow.html)__ and the second one spend most of the time converting array data to PNG in __[PIL.Image.save()](https://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.save)__.

But converting to PNG is not the fastest and only give me 2-3 FPS. If I use JPEG instead, framerate goes up to 36 FPS, which is not bad.

In [1]:
# Import the required modules
import cv2
import time
import numpy as np
import PIL.Image
from io import BytesIO
import IPython.display
import ipywidgets as widgets
import threading

In [2]:
#Use 'jpeg' instead of 'png' (~5 times faster)
def showarray(a, prev_display_id=None, fmt='jpeg'):
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    obj = IPython.display.Image(data=f.getvalue())
    if prev_display_id is not None:
        IPython.display.update_display(obj, display_id=prev_display_id)
        return prev_display_id
    else:
        return IPython.display.display(obj, display_id=True)

In [3]:
def get_frame(cam):
    # Capture frame-by-frame
    ret, frame = cam.read()
    
    #flip image for natural viewing
#     frame = cv2.flip(frame, 1)
    
    return frame

In [4]:
def test_cams(usb_cameras):
    for usb_camera in usb_cameras:
#         print(index)
        cap = cv2.VideoCapture()
        cap.open(usb_camera.get('path'))
        if cap.isOpened():
            print("active: ", usb_camera.get('path'))
        else:
            print("inactive: ", usb_camera.get('path'))
        cap.release()

In [5]:
usb_cameras = [
    {
        "path": "/dev/zuppacamera0",
        "name": "zuppacamera0"
    },
    {
        "path": "/dev/zuppacamera1",
        "name": "zuppacamera1"
    }
]

print(usb_cameras)
test_cams(usb_cameras)

[{'path': '/dev/zuppacamera0', 'name': 'zuppacamera0'}, {'path': '/dev/zuppacamera1', 'name': 'zuppacamera1'}]
active:  /dev/zuppacamera0
active:  /dev/zuppacamera1


In [6]:
cameras = []

def init_cameras():
    
    for usb_camera in usb_cameras:

        cam = cv2.VideoCapture(usb_camera.get('path'))

#         cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
#         cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
        cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
        cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
        
        cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
        cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

#         cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
#         cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)


    #     cam.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.25) #ref: https://github.com/opencv/opencv/issues/9738#issuecomment-346584044
    #     cam.set(cv2.CAP_PROP_EXPOSURE, 0.01)
    #     cam.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0)
    #     cam.set(cv2.CAP_PROP_EXPOSURE, -4.0)

        cameras.append({
            "name": usb_camera.get('name'),
            'cam': cam,
            'display_id': None,
        })
        
def stop_cameras():      
    for camera in cameras:
        cam = camera.get('cam')
        cam.release()

In [7]:
init_cameras()

In [8]:
from datetime import datetime
import os

button = widgets.Button(description="Capture All")
button_output = widgets.Output()

display(button, button_output)

main_dir = './captured'
sub_dir = '/tests'
final_dir = main_dir+sub_dir

if not os.path.exists(final_dir):
    os.makedirs(final_dir)

def on_button_clicked(b):
    with button_output:
        
#         print("Capturing...")
#         IPython.display.clear_output(wait=True)

        datetime_str = datetime.today().strftime('%Y-%m-%d_%H:%M:%S')
    
        for camera_num, camera in enumerate(cameras):

            name = camera.get('name')
            cam = camera.get('cam')
            for x in range(6): #lame way to clear image buffer
                frame = get_frame(cam)
            
            final_name = datetime_str+'_camera_'+str(name)+'.jpg'
            cv2.imwrite(final_dir + '/' + final_name, frame) #need to create folder called captures first!


            # Convert the image from OpenCV BGR format to matplotlib RGB format
            # to display the image
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            display_id = camera.get('display_id')
            if display_id is not None:
                showarray(frame, display_id)
            else:
                display_handle = showarray(frame)
                camera['display_id'] = display_handle.display_id
                

#         print("Captured.")
#         IPython.display.clear_output(wait=True)
        

button.on_click(on_button_clicked)

Button(description='Capture All', style=ButtonStyle())

Output()

In [9]:

# stop_cameras()        
    
# print ("Stream stopped")